In [1]:
import pandas as pd
import numpy as np

## Extraction Process

### ELO

In [2]:
ELO_file = "Resources/ELO.csv"
ELO_df = pd.read_csv(ELO_file)
ELO_df.head()

,ELO,TEAM,CONF
0,1770,Golden State Warriors,West
1,1661,San Antonio Spurs,West
2,1636,Los Angeles Clippers,West
3,1617,Utah Jazz,West
4,1602,Houston Rockets,West


### Endorsements

In [3]:
Endorsements_file = "Resources/Endorsements.csv"
Endorsements_df = pd.read_csv(Endorsements_file)
Endorsements_df.head()

,NAME,TEAM,SALARY,ENDORSEMENT
0,LeBron James,Cleveland Cavaliers,"$30,963,450.00","$55,000,000.00"
1,Kevin Durant,Golden State Warriors,"$26,500,000.00","$36,000,000.00"
2,Stephen Curry,Golden State Warriors,"$12,112,359.00","$35,000,000.00"
3,James Harden,Houston Rockets,"$26,500,000.00","$20,000,000.00"
4,Russell Westbrook,Oklahoma City Thunder,"$26,500,000.00","$15,000,000.00"


### Twitter

In [4]:
Twitter_file = "Resources/Twitter.csv"
Twitter_df = pd.read_csv(Twitter_file)
Twitter_df.head()

,PLAYER,TWITTER_FAVORITE_COUNT,TWITTER_RETWEET_COUNT
0,Russell Westbrook,2130.5,559.0
1,James Harden,969.0,321.5
2,Isaiah Thomas,467.5,155.5
3,Anthony Davis,368.0,104.0
4,DeMar DeRozan,0.0,186.0


### Wikipedia

In [5]:
Wiki_file = "Resources/Wikipedia.csv"
Wiki_df = pd.read_csv(Wiki_file)
Wiki_df.head()

,Unnamed: 0,names,pageviews,timestamps,wikipedia_handles
0,0,Russell Westbrook,3400,2016010100,Russell_Westbrook
1,1,Russell Westbrook,2893,2016010200,Russell_Westbrook
2,2,Russell Westbrook,3209,2016010300,Russell_Westbrook
3,3,Russell Westbrook,2531,2016010400,Russell_Westbrook
4,4,Russell Westbrook,2599,2016010500,Russell_Westbrook


## Transform Process

### ELO

In [6]:
ELOsorted_df=ELO_df.sort_values(['CONF','ELO'], ascending=False).rename(columns={'CONF':'CONFERENCE'})
ELOcleaned_df = ELOsorted_df.set_index('CONFERENCE')
ELOcleaned_df

,ELO,TEAM
CONFERENCE,,
West,1770,Golden State Warriors
West,1661,San Antonio Spurs
West,1636,Los Angeles Clippers
West,1617,Utah Jazz
West,1602,Houston Rockets
West,1563,Portland Trail Blazers
West,1552,Denver Nuggets
West,1543,Oklahoma City Thunder
West,1482,Memphis Grizzlies


### Endorsements

In [7]:
# Transforming Endorsements_df
Endorsements_df[Endorsements_df.columns[2:]] = Endorsements_df[Endorsements_df.columns[2:]].replace('[\$,]', '', regex=True).astype(float)

# Sorting values based on Endorsement first and then Salary
Cleaned_Endorsements = Endorsements_df.sort_values(['ENDORSEMENT', 'SALARY'], ascending=False)

# Cleaning up the formatting
Cleaned_Endorsements['SALARY'] = Cleaned_Endorsements['SALARY'].map("${:,.2f}".format)
Cleaned_Endorsements['ENDORSEMENT'] = Cleaned_Endorsements['ENDORSEMENT'].map("${:,.2f}".format)

Cleaned_Endorsements

,NAME,TEAM,SALARY,ENDORSEMENT
0,LeBron James,Cleveland Cavaliers,"$30,963,450.00","$55,000,000.00"
1,Kevin Durant,Golden State Warriors,"$26,500,000.00","$36,000,000.00"
2,Stephen Curry,Golden State Warriors,"$12,112,359.00","$35,000,000.00"
3,James Harden,Houston Rockets,"$26,500,000.00","$20,000,000.00"
4,Russell Westbrook,Oklahoma City Thunder,"$26,500,000.00","$15,000,000.00"
6,Derrick Rose,New York Knicks,"$21,323,252.00","$14,000,000.00"
5,Dwyane Wade,Chicago Bulls,"$23,200,000.00","$13,000,000.00"
9,Kyrie Irving,Cleveland Cavaliers,"$17,638,063.00","$13,000,000.00"
7,Carmelo Anthony,New York Knicks,"$24,559,380.00","$8,000,000.00"
8,Chris Paul,Los Angeles Clippers,"$22,868,827.00","$8,000,000.00"


### Twitter

In [8]:
# Changed column names, sorted the values by Favorite count and dropped any NaN rows.
Cleaned_Twitter = Twitter_df.rename(columns={'TWITTER_FAVORITE_COUNT':'FAVORITE COUNT', 'TWITTER_RETWEET_COUNT':'RETWEET COUNT'}).sort_values('FAVORITE COUNT', ascending=False).dropna(how='any')

# Filtering players with greater than 100 Favorite Count and resetting index
Cleaned_Twitter = Cleaned_Twitter.loc[Cleaned_Twitter['FAVORITE COUNT']>100].reset_index().drop(columns='index')

Cleaned_Twitter

,PLAYER,FAVORITE COUNT,RETWEET COUNT
0,Stephen Curry,12278.0,2893.0
1,Joel Embiid,6852.5,2941.0
2,LeBron James,5533.5,1501.5
3,Kawhi Leonard,2701.5,716.5
4,Russell Westbrook,2130.5,559.0
...,...,...,...
60,Patty Mills,112.0,19.0
61,Kyle Korver,107.5,30.5
62,Nerlens Noel,105.0,64.0
63,Julius Randle,102.0,59.0


### Wikipedia

In [9]:
# Dropped unnecessary columns(timestamps, wikipedia_handles)
Cleaned_Wiki = Wiki_df[['names', 'pageviews']].rename(columns={'names':'Player', 'pageviews':'Views'})

# Grouping by name, summing the pageviews and sorting by count
Cleaned_Wiki = Cleaned_Wiki.groupby('Player').agg('sum').sort_values('Views', ascending=False)

# Filtering players with greater than 1000 views
Cleaned_Wiki = Cleaned_Wiki.loc[Cleaned_Wiki['Views']>1000]

# Formatting
Cleaned_Wiki['Views'] = Cleaned_Wiki['Views'].map("{:,.0f}".format)

Cleaned_Wiki

,Views
Player,
Stephen Curry,"10,407,255"
LeBron James,"8,823,785"
Kevin Durant,"3,865,014"
Kyrie Irving,"3,022,758"
Dwyane Wade,"2,694,345"
...,...
Joe Young,"1,563"
Mirza Teletovic,"1,488"
Jusuf Nurkic,"1,423"


## Loading into SQLite

In [10]:
from sqlalchemy import create_engine

### ELO

In [ ]:
engine = create_engine ('sqlite:///SQLite/ELO.sqlite', echo=False)
conn = engine.connect()

table = 'ELO'
ELOcleaned_df.to_sql(table, conn)

conn.close()

### Endorsements

In [ ]:
engine = create_engine ('sqlite:///SQLite/Endorsements.sqlite', echo=False)
conn = engine.connect()

table = 'Endorsements'
Cleaned_Endorsements.to_sql(table, conn)

conn.close()

### Twitter

In [ ]:
engine = create_engine ('sqlite:///SQLite/Twitter.sqlite', echo=False)
conn = engine.connect()

table = 'Twitter'
Cleaned_Twitter.to_sql(table, conn)

conn.close()

### Wikipedia

In [ ]:
engine = create_engine ('sqlite:///SQLite/Wikipedia.sqlite', echo=False)
conn = engine.connect()

table = 'Wikipedia'
Cleaned_Wiki.to_sql(table, conn)

conn.close()

### Combined Into 1 Database

In [28]:
engine = create_engine ('sqlite:///SQLite/NBA_Social_Power.sqlite', echo=False)
conn = engine.connect()

table1 = 'ELO'
ELOcleaned_df.to_sql(table1, conn)

table2 = 'Endorsements'
Cleaned_Endorsements.to_sql(table2, conn)

table3 = 'Twitter'
Cleaned_Twitter.to_sql(table3, conn)

table4 = 'Wikipedia'
Cleaned_Wiki.to_sql(table4, conn)

conn.close()

In [29]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import inspect
from sqlalchemy.orm import Session

In [30]:
# Create the connection engine
engine = create_engine("sqlite:///SQLite/NBA_Social_Power.sqlite")

In [31]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

In [32]:
# Collect the names of tables within the database
inspector.get_table_names()

['ELO', 'Endorsements', 'Twitter', 'Wikipedia']

In [36]:
# Query All Records in the the Database
data = engine.execute('SELECT * FROM Wikipedia')

for record in data:
    print(record)

('Stephen Curry', '10,407,255')
('LeBron James', '8,823,785')
('Kevin Durant', '3,865,014')
('Kyrie Irving', '3,022,758')
('Dwyane Wade', '2,694,345')
('Klay Thompson', '2,619,119')
('Russell Westbrook', '2,367,837')
('Draymond Green', '1,866,704')
('Seth Curry', '1,802,876')
('Carmelo Anthony', '1,712,575')
('Derrick Rose', '1,678,701')
('Steven Adams', '1,445,971')
('Vince Carter', '1,420,828')
('Kevin Love', '1,418,113')
('Jeremy Lin', '1,404,816')
('Kawhi Leonard', '1,363,949')
('James Harden', '1,337,869')
('Dwight Howard', '1,323,270')
('Dirk Nowitzki', '1,303,433')
('Kris Humphries', '1,235,839')
('Andre Iguodala', '1,180,151')
('Chris Paul', '1,162,601')
('Kyle Lowry', '1,130,770')
('Richard Jefferson', '1,080,208')
('Metta World Peace', '1,072,646')
('Tony Parker', '1,061,395')
('Pau Gasol', '1,039,035')
('Shaun Livingston', '1,007,061')
('Blake Griffin', '988,555')
('DeMar DeRozan', '984,237')
('Tristan Thompson', '955,613')
('Damian Lillard', '926,756')
('Karl-Anthony Towns'

In [18]:
conn.close()

## Loading into MongoDB

In [21]:
import pymongo

In [22]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database
db = client.NBA_Social_Power

### ELO

In [23]:
# Define ELO collection
collection = db['ELO']

# Convert it to a dictionary
ELO_dict = ELOcleaned_df.to_dict("records")

# Inserting the dictionary into collection
collection.insert_many(ELO_dict)

### Endorsements

In [24]:
# Define Endorsements collection
collection = db['Endorsements']

# Convert it to a dictionary
Endorsements_dict = Cleaned_Endorsements.to_dict("records")

# Inserting the dictionary into collection
collection.insert_many(Endorsements_dict)

### Twitter

In [25]:
# Define Twitter collection
collection = db['Twitter']

# Convert it to a dictionary
Twitter_dict = Cleaned_Twitter.to_dict("records")

# Inserting the dictionary into collection
collection.insert_many(Twitter_dict)

### Wikipedia

In [26]:
# Define Wikipedia collection
collection = db['Wikipedia']

# Convert it to a dictionary
Wiki_dict = Cleaned_Wiki.to_dict("records")

# Inserting the dictionary into collection
collection.insert_many(Wiki_dict)